In [1]:
%cd ..

/Users/isham993/Desktop/Programming-Tutorials/2023-Data-Science/ML-Projects/end-to-end-project-with-mlflow


In [2]:
from utils import *

In [3]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-3-231-221-215.compute-1.amazonaws.com:5000'


In [4]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str
    metric_file_name: Path

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name,
            metric_file_name = config.metric_file_name,
        )

        return model_trainer_config

In [6]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def train(self):
        with mlflow.start_run():
            # Setting up tags - developer and model
            mlflow.set_tag("model", "xgboost")
            mlflow.set_tag("developer", "isham")

            train_data = pd.read_csv(self.config.train_data_path)
            test_data = pd.read_csv(self.config.test_data_path)

            # Tracking dataset
            mlflow.log_param("train_data", self.config.train_data_path)
            mlflow.log_param("test_data", self.config.test_data_path)

            train_x = train_data.drop([self.config.target_column], axis=1)
            test_x = test_data.drop([self.config.target_column], axis=1)
            train_y = train_data[[self.config.target_column]]
            test_y = test_data[[self.config.target_column]]

            # Model Training
            lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
            lr.fit(train_x, train_y)
            
            # Predictions
            y_pred = lr.predict(test_x)

            # Model Evaluation
            (rmse, mae, r2) = self.eval_metrics(test_y, y_pred)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            # Tracking parameters and evaluation metric
            mlflow.log_param("alpha", self.config.alpha)
            mlflow.log_param("l1_ratio", self.config.l1_ratio)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2) 
            
            # Saving the model
            model_path = os.path.join(self.config.root_dir, self.config.model_name)
            joblib.dump(lr, model_path)

            # Saving the model in AWS and Registering model
            # mlflow.log_artifact(local_path=model_path, artifact_path="models_pickle")
            mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetModel")

In [7]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-11-02 16:57:33,861: INFO: helper: yaml file: config/config.yaml loaded successfully]
[2023-11-02 16:57:33,863: INFO: helper: yaml file: params.yaml loaded successfully]
[2023-11-02 16:57:33,865: INFO: helper: yaml file: schema.yaml loaded successfully]
[2023-11-02 16:57:33,865: INFO: helper: created directory at: artifacts]
[2023-11-02 16:57:33,866: INFO: helper: created directory at: artifacts/model_trainer]
[2023-11-02 16:57:36,056: INFO: helper: json file saved at: artifacts/model_trainer/metrics.json]


/Users/isham993/mambaforge/envs/gpu_torch/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


[2023-11-02 16:57:40,825: INFO: credentials: Found credentials in shared credentials file: ~/.aws/credentials]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/11/02 16:57:45 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 5
Created version '5' of model 'ElasticnetModel'.
